In [ ]:
import torch
import transformers
from transformers import pipeline
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
import os
from tqdm import tqdm
from fpdf import FPDF
import pandas as pd

In [ ]:
def summarizer(summarization, file):
  #Loading the transcripted files
  loader = UnstructuredFileLoader(file)
  document = loader.load()

  #Splitting the large documents into smaller chunks
  text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
  texts = text_splitter.split_documents(document)

  #Converting the document type into string and storing it in a list
  docs = []
  for i in range(0, len(texts)):
    docs.append(texts[i].page_content)

  #Summarizing each chunk of the document
  summaries = summarization(docs)
  summary = "".join(d['summary_text'] for d in summaries)

  return summary

In [ ]:
summarization = pipeline('summarization')

In [ ]:
info_all_podcasts = pd.read_csv(path_to_directory+'podcast.csv')
titles = info_all_podcasts['Show'].unique().tolist()

In [ ]:
for title in tqdm(titles):
  for f in tqdm(os.listdir(path_to_directory+title)):
    if 'mp3' not in f:
      for transcript in os.listdir(path_to_directory+title+'/'+f):
        summary = summarizer(summarization, path_to_directory+title+'/'+f+'/'+transcript)
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', size = 12)
        pdf.multi_cell(0, 10, txt = summary.encode('latin-1', 'replace').decode('latin-1'))
        pdf.output(path_to_directory+title+'/'+f+'/'+transcript.replace('.pdf', '_summary.pdf'))